In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from config import get_cfg
from op import Operation

cfg = get_cfg()

operation = Operation(cfg)

In [ ]:
operation.load(path='./exp/D-Mid2-Encoder-brains18=0410-013241/model.pt', model_type='Mid2-Encoder')

In [ ]:
operation.analyse_ent_var_dice_and_draw_scatter_tumor(mode='Dropout')

In [ ]:
operation.analyse_ent_var_dice_and_draw_scatter_tumor(mode='Ensemble')

In [ ]:
operation.get_picture_many(mode='Ensemble')

In [ ]:
operation.get_picture_many(mode='Dropout')


In [ ]:
operation.analyse_ent_var_dice_and_draw_scatter(mode='Dropout')

In [ ]:
operation.analyse_ent_var_dice_and_draw_scatter(mode='Ensemble')

In [ ]:

imgs, folder, slice_id = operation.val_data[0]
y_pred, entropy, dices, p_numpy = operation.inference(image=imgs[2], y_gt=imgs[3], mode='Dropout', return_p=True)

In [ ]:
y_gt = imgs[3].numpy()
print(y_gt.shape)

In [ ]:
m, n = 240*240, 9
L = np.zeros((m, n))
tmp = y_gt.reshape(m)
print(np.argmax(tmp))
print(L.shape)
for i in range(m):
    L[i, tmp[i]] = 1

In [ ]:
print(p_numpy)

In [ ]:
P = p_numpy.reshape(9, -1).swapaxes(0, 1)

In [ ]:
print(P.shape)

In [ ]:
index = 38767
print(p_numpy[:, int(index / 240), int(index % 240)])
print(P[index, :])
print(L[index, :])
# for i in range(m):
#     if L[i, 0] != 1:
#         print(i)

In [ ]:

%load_ext autoreload
%autoreload 2

%matplotlib inline

# 引入必要的库
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

n_classes = 9
y_test = L
y_score = P
# 计算每一类的ROC
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], thresholds = roc_curve(y_test[:, i], y_score[:, i])
    # print('#################################################################################')
    # print(fpr[i])
    # print(tpr[i])
    # print(thresholds)
    # print(y_score[:, i], np.min(y_score[:, i]), np.max(y_score[:, i]))
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area（方法二）
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area（方法一）
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
# Finally average it and compute AUC
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
lw=2
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

# colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
colors = [plt.cm.get_cmap('tab10', 10)(c)[:3] for c in range(10)]
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()